In [1]:
import pandas as pd
from pathlib import Path

import time
import re
import numpy as np
import mojimoji
import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import lxml

pd.set_option('display.max_columns', None)

In [2]:
read_dir_path = Path('/Users/masudaniwabinari/Desktop/hpb_scraping/data/地図用のデータフレーム')
read_files_path = read_dir_path.glob('*.csv')

_list=[]
for read_file_path in read_files_path:
    _df = pd.read_csv(read_file_path)
    _list.append(_df)
df = pd.concat(objs=_list, axis=0).reset_index(drop=True)
df

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,イルミナメニュー化の有無,Aujuaメニュー化の有無,緯度,経度
0,島根,松江,Rote【ロテ】,https://beauty.hotpepper.jp/slnH000404873/?cstt=1,島根県松江市東朝日町151-49,JR松江駅から徒歩5分 イオン松江の目の前です 敷地内に無料駐車場12台 Free Wi-Fi,4350,18,237,179,7,【カットクーポン♪】シャンプーカット5150円→4950円,4950,【夏カラー多数揃ってます♪】カット＋カラー ￥9570→￥9020～,9020,【矯正レベル自在！】ブライトシート（縮毛矯正）+カット ¥19890→¥19340～,19340,24,41,48,237,4.65,4.7,4.7,4.7,4.3,0,0,35.463655,133.068334
1,島根,松江,freedom belinda 松江学園店 【フリーダム ベリンダ】,https://beauty.hotpepper.jp/slnH000448248/?cstt=2,島根県松江市学園2丁目26-24,島根県松江市学園２丁目26-24,2200,10,15,161,8,クーポンは毎回使えます。ネット予約が無理でも電話でとれる場合があります。,0,☆最新導入☆【髪質改善】次世代トリートメント ￥7700,7700,【☆NEW☆艶髪に】カット＋カラー+髪質改善トリートメント ￥11000,11000,34,42,5,15,4.51,4.5,4.7,4.6,4.7,0,0,35.484177,133.069973
2,島根,松江,SMITH AVEDA,https://beauty.hotpepper.jp/slnH000594593/?cstt=3,島根県松江市玉湯町布志名637-87,山陰道松江玉造降りて1分！ 東京から飛行機で約1時間！東京から日帰りOK！,5000,10,97,209,5,【人気No.1】デザインカット＋アミノ酸ケアカラー＋髪質改善トリートメント,13000,【ブリーチなしで透明感】カット＋Wケアカラー＋トリートメント,18000,カット＋カラー＋高濃度水素コラーゲントリートメント,18000,15,24,23,97,4.61,4.8,4.6,4.7,4.3,0,0,35.434452,133.020194
3,島根,松江,freedom rita【フリーダム リタ】,https://beauty.hotpepper.jp/slnH000611691/?cstt=4,島根県松江市古志原1丁目9-21,松江工業高校付近、国道432号沿い、島根県環境保健公社向かい,2200,4,0,63,3,"【人気メニュー★】カット+オーガニックフルカラー ¥4,950",4950,"【天然ハーブ配合♪】オーガニックフルカラー ¥3,850",3850,"【まずはこれ！】デザインカット ¥2,750",2750,29,37,0,0,4.71,4.6,4.7,4.7,4.7,0,0,35.445511,133.073891
4,島根,松江,lamiell matsue 【ラミールマツエ】,https://beauty.hotpepper.jp/slnH000221002/?cstt=5,島根県松江市上乃木4丁目20-68,駐車場13台完備 無料Wi-fi完備 くにびき道路 檜山トンネルより南へ約６５０ｍ,3850,8,104,340,8,当日割！限定クーポン#ケアカラー+骨格補正カット￥12500→￥8000,8000,青戸美咲限定＃カット＋カラー￥12100→￥8000,8000,山内オリジナルハイライトカラー＋似合わせカット◎￥22000→￥16500,16500,30,26,44,104,4.52,4.6,4.6,4.5,4.3,0,1,35.444086,133.065449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4129,福山・尾道,福山駅前・三吉周辺,Chill hair,https://beauty.hotpepper.jp/slnH000454867/?cst...,広島県福山市東町2丁目3-15,穴吹調理製菓専門学校のすぐ近く。駐車場あります。,3500,1,0,50,1,NaN,0,NaN,0,NaN,0,0,16,0,0,4.92,4.7,4.8,4.9,4.9,0,0,34.492225,133.368702
4130,福山・尾道,福山駅前・三吉周辺,SOYU headspa by Palm 【ソユ】,https://beauty.hotpepper.jp/slnH000582727/?cst...,広島県福山市山手町2-2-17 ｴｸｾﾚﾝﾃA-101,山手町コスモスの向かい側,0,2,0,33,2,NaN,0,NaN,0,NaN,0,0,23,0,0,4.93,4.9,4.9,4.9,4.8,0,0,34.491976,133.332330
4131,福山・尾道,福山駅前・三吉周辺,Triangle,https://beauty.hotpepper.jp/slnH000042802/?cst...,広島県福山市三吉町4-2-2,【三吉町】ニチエー三吉店隣,5500,4,0,3,1,NaN,0,NaN,0,NaN,0,0,43,0,0,4.50,4.5,5.0,5.0,4.5,0,0,34.494330,133.374803
4132,福山・尾道,福山駅前・三吉周辺,Calin【カラン】,https://beauty.hotpepper.jp/slnH000235928/?cst...,広島県福山市木之庄町2-1-41-3,【木之庄町】オンリーワンから徒歩1分/駐車場有,4400,1,0,43,1,NaN,0,NaN,0,NaN,0,0,12,0,0,4.69,4.6,4.7,4.6,4.4,0,0,34.496622,133.354260


In [43]:
def get_lat_lon_from_address(address: str) -> dict:
    """
    address_lにlistの形で住所を入れてあげると、latlonsという入れ子上のリストで緯度経度のリストを返す関数。
    >>>>get_lat_lon_from_address(['東京都文京区本郷7-3-1','東京都文京区湯島３丁目３０−１'])
    [['35.712056', '139.762775'], ['35.707771', '139.768205']]
    """
    url = 'http://www.geocoding.jp/api/'
    payload = {"v": 1.1, 'q': address}
    r = requests.get(url, params=payload)
    ret = BeautifulSoup(r.content,'lxml')
    if ret.find('error'):
        # raise ValueError(f"Invalid address submitted. {address}")
        time.sleep(10)
        return [0,0]
    else:
        time.sleep(10) # sleep(10)は絶対必要
        lat = ret.find('lat').string
        lon = ret.find('lng').string
        location_list = [lat, lon]
    return location_list


In [14]:
for address in df['住所']:
    if df[df['住所']==address]['緯度'].values[0]==0.0:
        location_list = get_lat_lon_from_address(address)
        df.loc[df['住所']==address, '緯度'] = location_list[0]
        df.loc[df['住所']==address, '経度'] = location_list[1]
        print(df[df['住所']==address][['緯度', '経度']])
    else:
        pass

/Users/masudaniwabinari/Desktop/hpb_scraping/.env/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


    緯度 経度
510  0  0


In [38]:
df[df['緯度']==0.0]['住所']

3591    香川県善通寺市原田町998
3685     愛媛県松山市花園町3-3
Name: 住所, dtype: object

In [16]:
df['住所']

0                  島根県松江市東朝日町151-49
1                  島根県松江市学園2丁目26-24
2                島根県松江市玉湯町布志名637-87
3                  島根県松江市古志原1丁目9-21
4                 島根県松江市上乃木4丁目20-68
                   ...             
4129                広島県福山市東町2丁目3-15
4130    広島県福山市山手町2-2-17 ｴｸｾﾚﾝﾃA-101
4131                 広島県福山市三吉町4-2-2
4132             広島県福山市木之庄町2-1-41-3
4133               広島県福山市木之庄町3-3-20
Name: 住所, Length: 4134, dtype: object

In [23]:
get_lat_lon_from_address('香川県善通寺市原田町998')

/Users/masudaniwabinari/Desktop/hpb_scraping/.env/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


['34.250717', '133.794193']

In [22]:
df.loc[df['住所']=='広島県広島市東区若草町10-9L-ｱﾏﾂﾗﾋﾞﾙ1階', '住所'] = '広島県広島市東区若草町10-9'

In [30]:
df.loc[df['住所']=='香川県善通寺市原田町998-1', '住所'] = '香川県善通寺市原田町998'
df.loc[df['住所']=='愛媛県松山市花園町3-3 花園町棚橋ﾋﾞﾙ1F', '住所'] = '愛媛県松山市花園町3-3'

In [37]:
for address in df['住所']:
    if df[df['住所']==address]['緯度'].values[0]==0.0:
        location_list = get_lat_lon_from_address(address)
        df.loc[df['住所']==address, '緯度'] = location_list[0]
        df.loc[df['住所']==address, '経度'] = location_list[1]
        print(df[df['住所']==address][['緯度', '経度']])
    else:
        pass

In [44]:
_list = get_lat_lon_from_address('愛媛県松山市花園町3-3')

df.loc[df['住所']=='愛媛県松山市花園町3-3', '緯度'] = _list[0]
df.loc[df['住所']=='愛媛県松山市花園町3-3', '経度'] = _list[1]

In [45]:
_list = get_lat_lon_from_address('香川県善通寺市原田町998')

df.loc[df['住所']=='香川県善通寺市原田町998', '緯度'] = _list[0]
df.loc[df['住所']=='香川県善通寺市原田町998', '経度'] = _list[1]

In [46]:
df[df['緯度']==0.0]['住所']

Series([], Name: 住所, dtype: object)

In [47]:
df

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,イルミナメニュー化の有無,Aujuaメニュー化の有無,緯度,経度
0,島根,松江,Rote【ロテ】,https://beauty.hotpepper.jp/slnH000404873/?cstt=1,島根県松江市東朝日町151-49,JR松江駅から徒歩5分 イオン松江の目の前です 敷地内に無料駐車場12台 Free Wi-Fi,4350,18,237,179,7,【カットクーポン♪】シャンプーカット5150円→4950円,4950,【夏カラー多数揃ってます♪】カット＋カラー ￥9570→￥9020～,9020,【矯正レベル自在！】ブライトシート（縮毛矯正）+カット ¥19890→¥19340～,19340,24,41,48,237,4.65,4.7,4.7,4.7,4.3,0,0,35.463655,133.068334
1,島根,松江,freedom belinda 松江学園店 【フリーダム ベリンダ】,https://beauty.hotpepper.jp/slnH000448248/?cstt=2,島根県松江市学園2丁目26-24,島根県松江市学園２丁目26-24,2200,10,15,161,8,クーポンは毎回使えます。ネット予約が無理でも電話でとれる場合があります。,0,☆最新導入☆【髪質改善】次世代トリートメント ￥7700,7700,【☆NEW☆艶髪に】カット＋カラー+髪質改善トリートメント ￥11000,11000,34,42,5,15,4.51,4.5,4.7,4.6,4.7,0,0,35.484177,133.069973
2,島根,松江,SMITH AVEDA,https://beauty.hotpepper.jp/slnH000594593/?cstt=3,島根県松江市玉湯町布志名637-87,山陰道松江玉造降りて1分！ 東京から飛行機で約1時間！東京から日帰りOK！,5000,10,97,209,5,【人気No.1】デザインカット＋アミノ酸ケアカラー＋髪質改善トリートメント,13000,【ブリーチなしで透明感】カット＋Wケアカラー＋トリートメント,18000,カット＋カラー＋高濃度水素コラーゲントリートメント,18000,15,24,23,97,4.61,4.8,4.6,4.7,4.3,0,0,35.434452,133.020194
3,島根,松江,freedom rita【フリーダム リタ】,https://beauty.hotpepper.jp/slnH000611691/?cstt=4,島根県松江市古志原1丁目9-21,松江工業高校付近、国道432号沿い、島根県環境保健公社向かい,2200,4,0,63,3,"【人気メニュー★】カット+オーガニックフルカラー ¥4,950",4950,"【天然ハーブ配合♪】オーガニックフルカラー ¥3,850",3850,"【まずはこれ！】デザインカット ¥2,750",2750,29,37,0,0,4.71,4.6,4.7,4.7,4.7,0,0,35.445511,133.073891
4,島根,松江,lamiell matsue 【ラミールマツエ】,https://beauty.hotpepper.jp/slnH000221002/?cstt=5,島根県松江市上乃木4丁目20-68,駐車場13台完備 無料Wi-fi完備 くにびき道路 檜山トンネルより南へ約６５０ｍ,3850,8,104,340,8,当日割！限定クーポン#ケアカラー+骨格補正カット￥12500→￥8000,8000,青戸美咲限定＃カット＋カラー￥12100→￥8000,8000,山内オリジナルハイライトカラー＋似合わせカット◎￥22000→￥16500,16500,30,26,44,104,4.52,4.6,4.6,4.5,4.3,0,1,35.444086,133.065449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4129,福山・尾道,福山駅前・三吉周辺,Chill hair,https://beauty.hotpepper.jp/slnH000454867/?cst...,広島県福山市東町2丁目3-15,穴吹調理製菓専門学校のすぐ近く。駐車場あります。,3500,1,0,50,1,NaN,0,NaN,0,NaN,0,0,16,0,0,4.92,4.7,4.8,4.9,4.9,0,0,34.492225,133.368702
4130,福山・尾道,福山駅前・三吉周辺,SOYU headspa by Palm 【ソユ】,https://beauty.hotpepper.jp/slnH000582727/?cst...,広島県福山市山手町2-2-17 ｴｸｾﾚﾝﾃA-101,山手町コスモスの向かい側,0,2,0,33,2,NaN,0,NaN,0,NaN,0,0,23,0,0,4.93,4.9,4.9,4.9,4.8,0,0,34.491976,133.33233
4131,福山・尾道,福山駅前・三吉周辺,Triangle,https://beauty.hotpepper.jp/slnH000042802/?cst...,広島県福山市三吉町4-2-2,【三吉町】ニチエー三吉店隣,5500,4,0,3,1,NaN,0,NaN,0,NaN,0,0,43,0,0,4.50,4.5,5.0,5.0,4.5,0,0,34.49433,133.374803
4132,福山・尾道,福山駅前・三吉周辺,Calin【カラン】,https://beauty.hotpepper.jp/slnH000235928/?cst...,広島県福山市木之庄町2-1-41-3,【木之庄町】オンリーワンから徒歩1分/駐車場有,4400,1,0,43,1,NaN,0,NaN,0,NaN,0,0,12,0,0,4.69,4.6,4.7,4.6,4.4,0,0,34.496622,133.35426


In [48]:
df.to_csv('/Users/masudaniwabinari/Desktop/hpb_scraping/data/地図用のデータフレーム/concat/地図用データフレームマージ.csv', index=False)